## BME 인공지능 3조 (ASL 인식 모델 구현 - inference ver.)
손모양(미국 수화) 사진을 이용해 해당 수화의 의미(알파벳)를 알려주는 모델 구현

###### 본 모델의 학습은 BME 인공지능 실습 자료인 ch06_SimpleConvNet.ipynb 을 참고/수정하여 이루어졌음을 밝힙니다.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torchvision.models as models

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# data iter

batch_size = 26
path = 'example Folder'

train_transforms = transforms.Compose(
      [
          transforms.Resize((64,64)),
          transforms.ToTensor(),
      ]
    )

pic = torchvision.datasets.ImageFolder(root = path,transform=train_transforms)

pic_iter = torch.utils.data.DataLoader(pic,batch_size=batch_size,shuffle=False)

# 모델 불러오기
net = models.resnext50_32x4d()

net.fc = torch.nn. Linear(in_features=2048, out_features=26, bias=True)

net.layer2 = torch.nn.Sequential(
    net.layer2[0],
    net.layer2[1],
    net.layer2[2],
    net.layer2[3],
    torch.nn.Dropout(0.2)
)

net.layer3 = torch.nn.Sequential(
    net.layer3[0],
    net.layer3[1],
    net.layer3[2],
    net.layer3[3],
    net.layer3[4],
    net.layer3[5],
    torch.nn.Dropout(0.2)
)

net.layer4 = torch.nn.Sequential(
    net.layer4[0],
    net.layer4[1],
    torch.nn.Dropout(0.02),
    net.layer4[2]
)
net.to(device)

net.load_state_dict(torch.load("BME Team3.params"))

# 데이터에 대한 결과확인
batch_size = 26
tgtnames = pic.classes

y_list = np.array([])
y_hat_list = np.array([])
for X,y in pic_iter:
    X=X.to(device)
    y_hat = net(X)    
    y_hat = y_hat.argmax(dim=1)
    
    y_list = np.append(y_list,y)
    y_hat_list = np.append(y_hat_list,y_hat.cpu().numpy())
    
    for n in range(len(y_hat)):
        plt.figure(dpi=64)
        plt.imshow(X[n].T.cpu())
        plt.title(' label:%s \n prediction:%s '%(tgtnames[y[n]],tgtnames[y_hat[n]]))
        plt.pause(.0001)